In [229]:
import pandas as pd
import numpy as np

In [256]:
df = pd.read_csv("diabetic_data.csv")
ids = pd.read_csv("IDS_mapping.csv")
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [257]:
df["time_in_hospital"].unique()

array([ 1,  3,  2,  4,  5, 13, 12,  9,  7, 10,  6, 11,  8, 14])

In [258]:
df["diag_1"].nunique()

717

## 1. Group Diagnosis Categories

#### ICD-9 categories 

0 : Invalid or Other

1 : Infectious and Parasitic Diseases

2 : Neoplasms

3 : Endocrine, Nutritional, Metabolic Diseases, and Immunity Disorders

4 : Diseases of the Blood and Blood-forming Organs

5 : Mental Disorders

6 : Diseases of the Nervous System and Sense Organs

7 : Diseases of the Circulatory System

8 : Diseases of the Respiratory System

9 : Diseases of the Digestive System

10 : Diseases of the Genitourinary System

11 : Complications of Pregnancy, Childbirth, and the Puerperium

12 : Diseases of the Skin and Subcutaneous Tissue

13 : Diseases of the Musculoskeletal System and Connective Tissue

14 : Congenital Anomalies

15 : Certain Conditions Originating in the Perinatal Period

16 : Symptoms, Signs, and Ill-defined Conditions

17 : Injury and Poisoning

18 : V-codes (where a patient is seeking health services but is not currently sick)


In [259]:
df["diag_1"].unique()

array(['250.83', '276', '648', '8', '197', '414', '428', '398', '434',
       '250.7', '157', '518', '999', '410', '682', '402', '737', '572',
       'V57', '189', '786', '427', '996', '277', '584', '462', '473',
       '411', '174', '486', '998', '511', '432', '626', '295', '196',
       '250.6', '618', '182', '845', '423', '808', '250.4', '722', '403',
       '250.11', '784', '707', '440', '151', '715', '997', '198', '564',
       '812', '38', '590', '556', '578', '250.32', '433', 'V58', '569',
       '185', '536', '255', '250.13', '599', '558', '574', '491', '560',
       '244', '250.03', '577', '730', '188', '824', '250.8', '332', '562',
       '291', '296', '510', '401', '263', '438', '70', '250.02', '493',
       '642', '625', '571', '738', '593', '250.42', '807', '456', '446',
       '575', '250.41', '820', '515', '780', '250.22', '995', '235',
       '250.82', '721', '787', '162', '724', '282', '514', 'V55', '281',
       '250.33', '530', '466', '435', '250.12', 'V53', '789', '

In [260]:
def categorize_icd9(code):

    if code == '?':
        return "None"

     # Category for V-codes
    if code.startswith('V') or code.startswith('v'):
        return 18  
    # Category for E-codes
    elif code.startswith('E') or code.startswith('e'):
        return 17  

    try:
        code_int = int(float(code))
            
        # Categorize based on integer value
        if 1 <= code_int <= 139:
            return 1
        elif 140 <= code_int <= 239:
            return 2
        elif 240 <= code_int <= 279:
            return 3
        elif 280 <= code_int <= 289:
            return 4
        elif 290 <= code_int <= 319:
            return 5
        elif 320 <= code_int <= 389:
            return 6
        elif 390 <= code_int <= 459:
            return 7
        elif 460 <= code_int <= 519:
            return 8
        elif 520 <= code_int <= 579:
            return 9
        elif 580 <= code_int <= 629:
            return 10
        elif 630 <= code_int <= 679:
            return 11
        elif 680 <= code_int <= 709:
            return 12
        elif 710 <= code_int <= 739:
            return 13
        elif 740 <= code_int <= 759:
            return 14
        elif 760 <= code_int <= 779:
            return 15
        elif 780 <= code_int <= 799:
            return 16
        elif 800 <= code_int <= 999:
            return 17
        else:
            return 0
    except ValueError:
        return 0


In [261]:
# Function to add categorized ICD-9 codes as a new column
def add_icd_cat(df, input_column, output_column):
    df[output_column] = df[input_column].apply(categorize_icd9)
    return output_column


In [262]:
add_icd_cat(df, 'diag_1', 'diag1_category')
add_icd_cat(df, 'diag_2', 'diag2_category')
add_icd_cat(df, 'diag_3', 'diag3_category')

'diag3_category'

In [263]:
df["diag1_category"].value_counts()


diag1_category
7       30336
3       11459
8       10407
9        9208
16       7636
17       6975
10       5078
13       4957
2        3433
1        2768
12       2530
5        2262
18       1644
6        1211
4        1103
11        687
14         51
None       21
Name: count, dtype: int64

In [264]:
print("? detected: 21")

print("Diag 1: ")
print(21/101766)

print("Diag 2: ")
print(358/101766)

print("Diag 3: ")
print(1423/101766)


? detected: 21
Diag 1: 
0.0002063557573256294
Diag 2: 
0.0035178743391702533
Diag 3: 
0.013983059174970029


## 2. Group Admission Type Categories

#### Admission Type Categories

0 : Not Available

1 : Emergency

2 : Urgent

3 : Elective

4 : Newborn

5 : Trauma Center


In [265]:
def clean_ad_type(df, input_column, output_column):
    df[output_column] = df[input_column]
    df[output_column] = df[output_column].replace([5, 6, 8], 0)
    df[output_column] = df[output_column].replace(7, 5)
    return output_column


In [266]:
clean_ad_type(df, 'admission_type_id', 'admission_type_cleaned')


'admission_type_cleaned'

In [267]:
df["admission_type_cleaned"].value_counts()

admission_type_cleaned
1    53990
3    18869
2    18480
0    10396
5       21
4       10
Name: count, dtype: int64

In [268]:
print(10396/101766)

0.1021559263408211


## 3. Group Admission Source Categories

#### Admission Source Categories

0 : Uncategorized/Unknown

Not Available

NULL

Not Mapped

Unknown/Invalid


1 : Referrals 

Physician Referral

Clinic Referral

HMO Referral


2 : Transfers from Healthcare Facilities

Transfer from a hospital

Transfer from a Skilled Nursing Facility (SNF)

Transfer from another health care facility

Transfer from critical access hospital

Transfer From Another Home Health Agency

Transfer from hospital inpt/same fac result in a sep claim

Transfer from Ambulatory Surgery Center

Transfer from Hospice


3 : Emergency Room


4 : Court/Law Enforcement


5 : Births and Related Admissions

Normal Delivery

Premature Delivery

Sick Baby

Extramural Birth

Born inside this hospital

Born outside this hospital


6 : Readmission to Same Home Health Agency


In [269]:
def clean_ad_source(df, input_column, output_column):
    df[output_column] = df[input_column]
    df[output_column] = df[output_column].replace([1, 2, 3], 1)
    df[output_column] = df[output_column].replace([4, 5, 6, 10, 18, 22, 25, 26], 2)
    df[output_column] = df[output_column].replace(7, 3)
    df[output_column] = df[output_column].replace(8, 4)
    df[output_column] = df[output_column].replace([11, 12, 13, 14, 23, 24], 5)
    df[output_column] = df[output_column].replace(19, 6)
    df[output_column] = df[output_column].replace([9, 15, 17, 20, 21], 0)
    return output_column

In [270]:
clean_ad_source(df, 'admission_source_id', 'admission_source_cleaned')


'admission_source_cleaned'

In [271]:
df["admission_source_cleaned"].value_counts()

admission_source_cleaned
3    57494
1    30856
0     7067
2     6328
4       16
5        5
Name: count, dtype: int64

In [272]:
print(7067/101766)

0.06944362557239156


### Check for missing values (max_glu_serum)

In [273]:
df["max_glu_serum"].unique()

array([nan, '>300', 'Norm', '>200'], dtype=object)

In [274]:
df["max_glu_serum"].value_counts()

max_glu_serum
Norm    2597
>200    1485
>300    1264
Name: count, dtype: int64

In [275]:
print(df["max_glu_serum"].isna().sum())

96420/101766

96420


0.9474677200636755

In [276]:
df['max_glu_serum_cleaned'] = df['max_glu_serum'].fillna("None")


In [277]:
df['max_glu_serum_cleaned'].value_counts()

max_glu_serum_cleaned
None    96420
Norm     2597
>200     1485
>300     1264
Name: count, dtype: int64

## Save new columns as CSV

In [278]:
new = df[["admission_type_cleaned", "admission_source_cleaned", "diag1_category", "diag2_category", "diag3_category", "max_glu_serum_cleaned"]]
new



,admission_type_cleaned,admission_source_cleaned,diag1_category,diag2_category,diag3_category,max_glu_serum_cleaned
0,0,1,3,None,None,None
1,1,3,3,3,3,None
2,1,3,11,3,18,None
3,1,3,1,3,7,None
4,1,3,2,2,3,None
...,...,...,...,...,...,...
101761,1,3,3,5,7,None
101762,1,2,9,3,16,None
101763,1,3,1,10,5,None
101764,2,3,17,4,17,None


In [283]:
new.to_csv("cleaned_columns.csv")